In [1]:
import random
import numpy as np
import torch

model_name = "google/mt5-base"

special_token = '<sep>'
global_prompt_pattern = f"%s Was ist die Definition von %s? "
configs = [
    # "seq_bn",
    # "double_seq_bn",
    # "par_bn",
    # "scaled_par_bn",
    # "seq_bn_inv",
    # "double_seq_bn_inv",
    # "compacter",
    # "compacter++",
    # "prefix_tuning",
    # "prefix_tuning_flat",
    # "ia3",
    "mam",
    # "unipelt",
   # "prompt_tuning"
]

train_len, val_len = 1248, 154
train_len, val_len = -1, -1
# train_len, val_len = 50, 100
epochs = 20
logging_steps = 200
bf16 = True

# Set the random seed for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
seed = 42
set_seed(seed)

In [2]:
import sys
sys.path.insert(0, '../')
from datasets import Dataset
from src.training import generator
from src.utils import sanitize_context_word, sanitize_context
from src.mlflow_utils import mlflow
from src.prompting import prompt_pattern
from transformers import T5Tokenizer
import re
import datetime

mlflow.set_experiment(experiment_id=7)

# prefix = "define: "
tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
# tokenizer.add_tokens(['ä', 'Ä', 'ö', 'Ö', 'ü', 'Ü', 'ß', 'ẞ'])

# Add special token if needed
# if special_token not in tokenizer.get_vocab():
#     tokenizer.add_tokens([special_token])
    

def sanitize(input_):
    input_ = input_.replace("''", "")
    input_ = re.sub('\s+', ' ', re.sub('\n+', ' ', input_.strip()))
    input_ = sanitize_context(input_)
    return input_

def preprocess(examples):
    input_texts = [prompt_pattern(sanitize(context), sanitize_context_word(word), pattern=global_prompt_pattern) for context, word in zip(examples["context_sentence"], examples["context_word"])]
    inputs = tokenizer(input_texts, max_length=512, truncation=True)
    inputs["labels"] = tokenizer(text_target=[sanitize(doc) for doc in examples["gt"]], max_length=128, truncation=True)["input_ids"]
    inputs["debug_text"] = input_texts
    inputs["debug_gt"] = [sanitize(doc) for doc in examples["gt"]]

    return inputs


dataset_train = Dataset.from_parquet("../dataset/v1/train.parquet", split="train").shuffle(seed=42)
dataset_val = Dataset.from_parquet("../dataset/v1/val.parquet", split="val").shuffle(seed=42)

if train_len != -1:
    dataset_train = dataset_train.select(range(train_len))
if val_len != -1:
    dataset_val = dataset_val.select(range(val_len))

print(f"Train: {len(dataset_train)} - Val: {len(dataset_val)}")

# Encode the input data
dataset_train = dataset_train.map(preprocess, batched=True)
# dataset_train = dataset_train.rename_column(original_column_name="label", new_column_name="labels")
# # Transform to pytorch tensors and only output the required columns
# dataset_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Encode the input data
dataset_val = dataset_val.map(preprocess, batched=True)
# # The transformers model expects the target class column to be named "labels"
# dataset_val = dataset_val.rename_column(original_column_name="label", new_column_name="labels")
# # Transform to pytorch tensors and only output the required columns
# dataset_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Train: 124852 - Val: 15432


In [3]:
from termcolor import colored

def test_inference(model, index, dataset=dataset_train, skip_special_tokens=False):
    gen_config = GenerationConfig(max_new_tokens=128, do_sample=False)
    gen_config = GenerationConfig(
        max_length=50, 
        num_beams=5, 
        early_stopping=True
    )

    datapoint = dataset[index]
    
    # input_text = f"{sanitize_context(datapoint['input'][2])} Was ist die Definition von \"{sanitize_context_word(datapoint['input'][1])}\"?"
    input_text = prompt_pattern(datapoint["context_sentence"], sanitize_context_word(datapoint['context_word']), pattern=global_prompt_pattern)
    input_ids = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(input_ids['input_ids'].to('cuda'), generation_config=gen_config)
    
    return "Prompt: " + tokenizer.decode(datapoint['input_ids'], skip_special_tokens=skip_special_tokens) + "\n" + colored("Prediction: " + tokenizer.decode(outputs[0], skip_special_tokens=skip_special_tokens), "yellow") + "\n" + colored("Ground-Truth: " + tokenizer.decode(datapoint['labels'], skip_special_tokens=skip_special_tokens), "green")

In [4]:
from transformers import TrainerCallback
import ipywidgets as widgets

class EvalCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        response = test_inference(model, 0)
        out.clear_output()
        with out:
            print(response)
        return control

out = widgets.Output(layout={'border': '1px solid black'})
out

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [9]:
from transformers import MT5ForConditionalGeneration, EarlyStoppingCallback
import adapters
from adapters import setup_adapter_training, AdapterArguments, AutoAdapterModel, T5AdapterModel

from tqdm.auto import tqdm

for adapter_config in tqdm(configs):
    with mlflow.start_run():
        print(adapter_config)
        model = MT5ForConditionalGeneration.from_pretrained(model_name)
        model.resize_token_embeddings(len(tokenizer))
        # model = AutoAdapterModel.from_pretrained(model_name)  # type: T5AdapterModel
        adapters.init(model)
        adapter_name = adapter_config
        adapter_args = AdapterArguments(train_adapter=True, adapter_config=adapter_config)
        setup_adapter_training(model, adapter_args, adapter_name)
        mlflow.log_param("base_model", model_name)
        mlflow.log_param("adapter_config", adapter_args.adapter_config)
        mlflow.log_param("prompt_pattern", global_prompt_pattern)

        # Log the Dataset to an MLflow run by using the `log_input` API
        mlflow.log_input(mlflow.data.from_huggingface(dataset_train, targets='debug_gt'), context="training")
        mlflow.log_input(mlflow.data.from_huggingface(dataset_val, targets='debug_gt'), context="validation")
        # model.add_adapter(adapter_name)
        # model.add_seq2seq_lm_head(adapter_name)
        
        # model.load_adapter("./example_test_2/")
        
        # print(list(model.adapters_config))
        # model.train_adapter(adapter_name)
        # model.set_active_adapters(adapter_name)
        
        from transformers import GenerationConfig
        import torch
        
        print(model.num_parameters(only_trainable=True))
        print(model.num_parameters(only_trainable=False))
        
        
        from src.ha_utils import HassioCallback
        from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
        from adapters import AdapterTrainer
        
        data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)
        
        training_args = Seq2SeqTrainingArguments(
            learning_rate=1e-4,
            evaluation_strategy="steps",
            num_train_epochs=epochs,
            weight_decay=0.01,
            per_device_train_batch_size=12,
            per_device_eval_batch_size=32,
            logging_steps=10,
            output_dir="./training_output",
            overwrite_output_dir=True,
            remove_unused_columns=True,
            predict_with_generate=True,
            eval_accumulation_steps=1,
            eval_steps=500,
            bf16=bf16,
            load_best_model_at_end=True,
            metric_for_best_model="loss",
            seed=seed
        )
        
        trainer = AdapterTrainer(
            model=model,
            args=training_args,
            train_dataset=dataset_train,
            eval_dataset=dataset_val,
            # compute_metrics=compute_accuracy,
            data_collator=data_collator,
            tokenizer=tokenizer,
            callbacks=[HassioCallback, EarlyStoppingCallback(early_stopping_patience=10)] #, EvalCallback]
        )
    
        trainer.train()

        adapter_path = "./Adapters_Experiments/" + f"{adapter_config}_{mlflow.active_run().info.run_id}_{mlflow.active_run().info.run_name}"
        model.save_adapter(adapter_path, adapter_name)
        mlflow.log_artifact(adapter_path)
        
model.eval()

print(test_inference(model, 0))

  0%|          | 0/1 [00:00<?, ?it/s]

mam


/home/jfeil/.pyenv/versions/3.10.12/envs/MasterThesis-NLP/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
2024/06/25 23:19:18 WARNING mlflow.data.huggingface_dataset: Failed to infer schema for Hugging Face dataset. Exception: Failed to infer schema for pandas.Series 0       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
2       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
3       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
4       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
                              ...                        
9995    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
9996    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
9997    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
9998    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

60390240
642773088


Step,Training Loss,Validation Loss
500,4.921000,4.340755
1000,4.539600,4.083334
1500,4.511200,3.981219
2000,4.276700,3.918077
2500,4.324700,3.880519
3000,4.213100,3.841833
3500,4.247800,3.793316
4000,4.068200,3.769759
4500,4.203700,3.737680
5000,3.895000,3.722696


Overwriting existing adapter 'mam'.


Prompt: Das alte Schiff wurde mit einem Schlepptau zurück in den Hafen gebracht. Was ist die Definition von Schlepptau?</s>
Prediction: <pad> eine Stadt in Nordrhein-Westfalen, Deutschland</s>
Ground-Truth: starkes Seil, das zum Ziehen eines Fahrzeuges verwendet wird</s>


In [ ]:
model.save_adapter("./Adapters_Experiments/" + adapter_config, adapter_name)
print(test_inference(model, 0))

In [5]:
from transformers import MT5ForConditionalGeneration
import adapters
from adapters import setup_adapter_training, AdapterArguments, AutoAdapterModel, T5AdapterModel

from tqdm.auto import tqdm

for adapter_config in tqdm(configs):
    with mlflow.start_run():
        print("FULL FINE-TUNING")
        model = MT5ForConditionalGeneration.from_pretrained(model_name)
        model.resize_token_embeddings(len(tokenizer))
        from transformers import GenerationConfig
        import torch
        
        print(model.num_parameters(only_trainable=True))
        print(model.num_parameters(only_trainable=False))
        
        
        from src.ha_utils import HassioCallback
        from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
        from adapters import AdapterTrainer
        
        data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)
        
        training_args = Seq2SeqTrainingArguments(
            learning_rate=1e-4,
            evaluation_strategy="steps",
            num_train_epochs=epochs,
            weight_decay=0.01,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=32,
            logging_steps=logging_steps,
            output_dir="./training_output",
            overwrite_output_dir=True,
            remove_unused_columns=True,
            predict_with_generate=True,
            eval_accumulation_steps=1,
            eval_steps=logging_steps,
            bf16=bf16
        )
        
        trainer = Seq2SeqTrainer(
            model=model,
            args=training_args,
            train_dataset=dataset_train,
            eval_dataset=dataset_val,
            # compute_metrics=compute_accuracy,
            data_collator=data_collator,
            tokenizer=tokenizer,
            callbacks=[HassioCallback, EvalCallback]
        )
    
        trainer.train()
print(test_inference(model, 0))


  0%|          | 0/1 [00:00<?, ?it/s]

FULL FINE-TUNING
582401280
582401280


Step,Training Loss,Validation Loss
200,6.742000,3.826421
400,4.268400,3.584789
600,3.720400,3.560444
800,3.354200,3.568357
1000,3.056100,3.556889
1200,2.866200,3.617332
1400,2.699800,3.623008
1600,2.529700,3.688983
1800,2.386100,3.719254
2000,2.309600,3.761853


Prompt: "Das alte Schiff wurde mit einem Schlepptau zurück in den Hafen gebracht.": Was ist die Definition von Schlepptau?</s>
Prediction: <pad> kleiner, geländegängiges Seil, mit dem man schwimmt, ohne Ende oder Antrieb auf einen bestimmten Weg gebracht wird</s>
Ground-Truth: starkes Seil, das zum Ziehen eines Fahrzeuges verwendet wird</s>


In [10]:
for i in range(min(10, len(dataset_train) if train_len == -1 else train_len)):
    print(test_inference(model, i, dataset_val))

Prompt: Politisch gliedert sich Long Island in die vier Teile Brooklyn, Queens, Nassau County und Suffolk County. Was ist die Definition von Long Island?</s>
Prediction: <pad> eine Stadt in Nordrhein-Westfalen, Deutschland</s>
Ground-Truth: Insel vor der Küste der USA, zu dessen Bundesstaat New York gehörig</s>
Prompt: Den Kranken ist überhaupt, wie auch Gerson richtig ausspricht, dringend zur Pflicht zu machen, sich nicht an eine bestimmte Modezahl von Bädern, wie sie vorgeschrieben zu werden pflegen, zu halten, sondern genau auf die an ihrem Körper beobachteten Reactionen zu achten. Was ist die Definition von Modezahl?</s>
Prediction: <pad> eine Stadt in Nordrhein-Westfalen, Deutschland</s>
Ground-Truth: Zahlenangabe, die keine gesicherte Quelle hat oder nicht mit einer anerkannten Methode gewonnenen wurde, die aber gleichwohl in der öffentlichen Diskussion verwendet wird</s>
Prompt: Bevor er sich an der Filmhochschule in München einschrieb, erlernte er das Sattlerhandwerk. Was ist d

In [17]:
model = T5ForConditionalGeneration.from_pretrained(model_name)
# model = AutoAdapterModel.from_pretrained(model_name)  # type: T5AdapterModel
adapters.init(model)
for config in configs:
    adapter_args = AdapterArguments(train_adapter=True, adapter_config=adapter_config)
    model.load_adapter(f"./Adapters_Experiments/{config}", load_as=config)
    # model.add_adapter(adapter_name)
    # model.add_seq2seq_lm_head(adapter_name)
    
    
    print(config)
    # model.set_active_adapters(adapter_name)
print(list(model.adapters_config))

seq_bn
double_seq_bn
par_bn
seq_bn_inv
double_seq_bn_inv
compacter
compacter++
prefix_tuning
prefix_tuning_flat
lora
ia3
mam
unipelt
['seq_bn', 'double_seq_bn', 'par_bn', 'seq_bn_inv', 'double_seq_bn_inv', 'compacter', 'compacter++', 'prefix_tuning', 'prefix_tuning_flat', 'lora', 'ia3', 'mam', 'unipelt']


In [8]:
dataset_train

Dataset({
    features: ['title', 'context_word', 'context_sentence', 'gt', 'input_ids', 'attention_mask', 'labels', 'debug_text', 'debug_gt'],
    num_rows: 124852
})

In [17]:
i.keys()

dict_keys(['title', 'context_word', 'context_sentence', 'gt', 'input_ids', 'attention_mask', 'labels', 'debug_text', 'debug_gt'])

In [21]:
for configs in config:
    model.set_active_adapters(config)
    test_inference(0)

Prompt: "Das alte Schiff wurde mit einem Schlepptau zur ü ck in den Hafen gebracht.": Was ist die Definition von Schlepptau?</s>
Prediction: <pad>f ü r die Schle ß f ü r einen Schlepp aus der Schle ß f ü r r einen Schleppschiff</s>
Ground-Truth: starkes Seil, das zum Ziehen eines Fahrzeuges verwendet wird</s>
Prompt: "Das alte Schiff wurde mit einem Schlepptau zur ü ck in den Hafen gebracht.": Was ist die Definition von Schlepptau?</s>
Prediction: <pad>f ü r die Schle ß f ü r einen Schlepp aus der Schle ß f ü r r einen Schleppschiff</s>
Ground-Truth: starkes Seil, das zum Ziehen eines Fahrzeuges verwendet wird</s>
Prompt: "Das alte Schiff wurde mit einem Schlepptau zur ü ck in den Hafen gebracht.": Was ist die Definition von Schlepptau?</s>
Prediction: <pad>f ü r die Schle ß f ü r einen Schlepp aus der Schle ß f ü r r einen Schleppschiff</s>
Ground-Truth: starkes Seil, das zum Ziehen eines Fahrzeuges verwendet wird</s>
Prompt: "Das alte Schiff wurde mit einem Schlepptau zur ü ck in den

In [31]:
for val in dataset_train:
    print(val)

{'title': 'Schlepptau', 'context_word': 'Schlepptau', 'context_sentence': "Das alte Schiff wurde mit einem ''Schlepptau'' zurück in den Hafen gebracht.", 'gt': 'starkes Seil, das zum Ziehen eines Fahrzeuges verwendet wird', 'input_ids': [96, 17266, 2105, 19447, 1177, 181, 665, 19461, 1572, 17, 402, 4204, 16, 177, 28290, 3, 10350, 535, 10, 2751, 229, 67, 15476, 193, 19461, 1572, 17, 402, 58, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [7133, 15, 7, 679, 173, 6, 211, 674, 3969, 15, 3225, 266, 7, 11786, 15, 7, 8533, 551, 1], 'debug_text': '"Das alte Schiff wurde mit einem Schlepptau zurück in den Hafen gebracht.": Was ist die Definition von Schlepptau?', 'debug_gt': 'starkes Seil, das zum Ziehen eines Fahrzeuges verwendet wird'}
{'title': 'fernhalten', 'context_word': 'fernhalten', 'context_sentence': "„1924 konnte die alte Schleuse das Hochwasser nicht von der Innenstadt ''fernhalten'', sodass von 1926 bis 19

In [30]:
for val in dataset_train:
    print(val['gt'])

starkes Seil, das zum Ziehen eines Fahrzeuges verwendet wird
etwas oder jemanden davon abhalten, an einen bestimmten Ort zu gelangen
ein durch Verkürzung eines Wortes/einer Wortverbindung entstandenes Wort; Abkürzung, Abbreviatur
Druckwerk
Hinweis; Andeutung; eine Information, die bestimmte Ereignisse wahrscheinlich macht


In [84]:
# model.save_adapter("./Adapters_Experiments/experiment-1", adapter_name)

In [164]:
test_inference(0, skip_special_tokens=False)

Prompt: "Das alte Schiff wurde mit einem Schlepptau zur ü ck in den Hafen gebracht.": Was ist die Definition von Schlepptau?</s>
Prediction: <pad>jemanden, die jemanden mit einem Schlepptau f ü rstig oder seinen Schlock f ü r den Schlock</s>
Ground-Truth: starkes Seil, das zum Ziehen eines Fahrzeuges verwendet wird</s>


In [132]:
for i in range(100):
    test_inference(i)

Prompt: "Das alte Schiff wurde mit einem Schlepptau zur ü ck in den Hafen gebracht.": Was ist die Definition von Schlepptau?</s>
Prediction: <pad>Schiff, die f ü r die Schiffe h ä lt mit seinen Schr ä ppen f ü r die Schiffe ern</s>
Ground-Truth: starkes Seil, das zum Ziehen eines Fahrzeuges verwendet wird
Prompt: "1924 konnte die alte Schleuse das Hochwasser nicht von der Innenstadt fernhalten, sodass von 1926 bis 1929 das heutige Sperrwerk errichtet wurde.": Was ist die Definition von fernhalten?</s>
Prediction: <pad>fernen, die sich nicht aus dem Erden f ü r f ü r seinen eigenen Körper f ü r seinen eigenen Körper f ü r seinen eigenen Körper f ü r seinen eigenen Körper f ü r seinen eigenen Körper f ü r seinen eigenen Körper f ü r seinen eigenen Körper f ü r seinen eigenen Körper f ü r seinen eigenen Körper f ü r seinen eigenen Körper f ü r seinen eigenen Körper f
Ground-Truth: etwas oder jemanden davon abhalten, an einen bestimmten Ort zu gelangen
Prompt: "V komunikaci se p<unk> iroze

KeyboardInterrupt: 

In [98]:
input_ids['input_ids'][0]

tensor([   96,   308,     9,   266,  2903,   501,    67,     3, 11150,   157,
         2014, 18199,   152, 24008,  5754,   311,    16, 26584, 26212,     6,
         4736,     3,    49,   181,  2907,    20, 10081,    15,     3, 22554,
           35,  2298,   535,    10,  2751,   229,    67, 15476,   193,    20,
        10081,    15,    58,     1])

In [100]:
outputs[0]

tensor([0, 3, 1], device='cuda:0')

In [25]:
for dat in dataset_train:
    print(tokenizer.decode(dat["labels"], skip_special_tokens=False))

Taxonomie Biologische Systematik (neulateinisch) Ordnung oder fachwissenschaftlich Ordo (Zusammenfassung mehrerer eng verwandter Familien, Teil eine Klasse)</s>
die Gestalt, das <unk> ußere, die Erscheinung</s>
militärische Abteilung, Kriegsflotte, Heer</s>
nächstkleinere Unterteilung der taxonomischen Regna (Reiche)</s>
Biologie die hierarchische Gliederungsstufe der Divisio (deutsch Abteilung) im Reich der Pflanzen und der Pilze wird weiter in Subdivisiones (deutsch Unterabteilungen) differenziert</s>
Biologie, Systematik fachwissenschaftlicher Terminus für das zoologische, hierarchisch hoch angesiedelte Taxon des Stammes, das zwischen dem Regnum (deutsch Reich) und der Classis (deutsch Klasse) steht. Im Pflanzenreich entspricht formal dem Phylum die Divisio (deutsch die Abteilung)., Seite 880&nbspf., Kapitel Systematik</s>
Biologie, Systematik fachwissenschaftlicher Terminus für das zoologische, hierarchisch hoch angesiedelte Taxon des Stammes, das zwischen dem Regnum (deutsch Reich

In [44]:
input_ids

tensor([[ 6634,    10,  7974,    74, 26082,    35,   736,    17,     7, 19107,
           736,  3272,  2499,   436,   319,   411,  3522,    32,    64,  2262,
          3484,    63,  9903, 30180,     5,  8262,     3,    23,  8919, 11589,
             9,    89,  9629,     9,     7,   229,   736,  3272,  2499,  2800,
          7537,     7,     3, 13392,    16, 22655,   587,  5704,    23,     9,
           401, 22093, 12711,  1923,     9, 17801,     3, 18007,   551,     6,
           211,   211,  1480,    77,  2014, 12503,   346,    17,  8533,     6,
             3,   547,   736,  3272,  2499,  1834,  7367, 19102,     5,  2751,
           229,    67, 15476,   193,    96,  8123,  3272,  2499,   121,    58,
             1]], device='cuda:0')

In [43]:
torch.tensor(datapoint['input_ids']).to('cuda')

tensor([ 6634,    10,  7974,    74, 26082,    35,   736,    17,     7, 19107,
          736,  3272,  2499,   436,   319,   411,  3522,    32,    64,  2262,
         3484,    63,  9903, 30180,     5,  8262,     3,    23,  8919, 11589,
            9,    89,  9629,     9,     7,   229,   736,  3272,  2499,  2800,
         7537,     7,     3, 13392,    16, 22655,   587,  5704,    23,     9,
          401, 22093, 12711,  1923,     9, 17801,     3, 18007,   551,     6,
          211,   211,  1480,    77,  2014, 12503,   346,    17,  8533,     6,
            3,   547,   736,  3272,  2499,  1834,  7367, 19102,     5,  2751,
          229,    67, 15476,   193,    96,  8123,  3272,  2499,   121,    58,
            1], device='cuda:0')

In [31]:
tokenizer.decode(datapoint['labels'], skip_special_tokens=True)

'semitische, genauer äthiosemitische Sprache der Amharen, die vor allem in thiopien und Eritrea gesprochen wird'

In [16]:
tokenizer.decode(dataset_val[200]['input_ids'], skip_special_tokens=True)

'Westfälische Kürassiere verfolgten die russischen Reiter. Was ist die Definition von "Kürassiere"?'

In [79]:
dataset_val[200]['gt']

'im 15. bis 19.\u2002Jahrhundert ein Soldat der schweren Reiterei der einen Kürass (Brustpanzer) trägt; neben den Lanzierern die älteste Gattung der frühneuzeitlichen Kavallerie'

In [78]:
sanitize_context(dataset_val[200]['gt'])

'im 15. bis 19.\u2002Jahrhundert ein Soldat der schweren Reiterei der einen Kürass (Brustpanzer) trägt neben den Lanzierern die älteste Gattung der frühneuzeitlichen Kavallerie'